<a href="https://colab.research.google.com/github/RafsanJany-44/ARC_Machine/blob/main/Tinnitus_PreTraining_Tunning_Hybride_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Function

In [141]:
!pip install imbalanced-learn
!pip install ipywidgets

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
import ipywidgets as widgets
from joblib import Parallel, delayed
from google.colab import files
import joblib
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


def dataset_balance(X_temp, y_temp):
  smote = SMOTE()
  X_temp, y_temp= smote.fit_resample(X_temp, y_temp)
  return pd.concat([pd.DataFrame(X_temp), pd.DataFrame(y_temp)], axis=1)


def Xy_balance(X_temp, y_temp):
  smote = SMOTE()
  return smote.fit_resample(X_temp, y_temp)


def save_model(model,model_name):
 joblib.dump(model, model_name)
  

def load_model(model_name):
  return joblib.load(model_name)


def Search_Null(dataset):
  dic={}
  for col in list(dataset.columns):
    rows = []
    flage = 0
    for row in range(dataset.shape[0]):
      if str(dataset[col][row]) == "nan":
        rows.append(row)
        flage = 1
    if flage ==1:
      dic[col] = rows
  return dic



def divide_dataset(dataset,percentage,random_state = 0):
  rows,cols = dataset.shape
  if random_state == 0:
    return dataset.iloc[:int(rows*(percentage)),:]
  else:
    import random
    l_r = random.sample(range(0, rows), int(rows*(percentage)))
    return dataset.iloc[l_r,:]



def StandardScaleData(data):
  scaler = StandardScaler()
  scaler.fit(data)
  return  scaler.transform(data)



def MinMaxScaleData(data):
  scaler = MinMaxScaler()
  scaler.fit(data)
  return scaler.transform(data)


def tune_report_csv(model,dic,feat_selection_method,number_of_feat):

  tunning_type = []
  accuracy = []
  scaled_accuracy = []
  model_name = []
  feature_selection_method = []
  number_of_features = []

  li = ["Smote : NO    ; Scaling : Train(Standard)",
      "Smote : NO    ; Scaling : Train(MinMax)",
      "Smote : Train ; Scaling : Train(Standard)",
      "Smote : Train ; Scaling : Train(MinMax)",
      "Smote : All   ; Scaling : Train(Standard)",
      "Smote : All   ; Scaling : Train(MinMax)"]

  for i in list(dic.keys()):
    X_train, X_test, y_train, y_test = dic[i]
    
    if i in li and str(model[:3]) == "xgb":
      model.fit(X_train,y_train.values)
      y_pred = model.predict(X_test.values)
    else:
      model.fit(X_train,y_train)
      y_pred = model.predict(X_test)

    accu =accuracy_score(y_test, y_pred)
    scaled_accu =  float("{:.2f}".format(accu))
    print(i)
    print("Accurecy: ",scaled_accu)
    tunning_type.append(i)
    accuracy.append(accu)
    scaled_accuracy.append(scaled_accu)
    model_name.append(model)
    feature_selection_method.append(feat_selection_method)
    number_of_features.append(number_of_feat)
    
    print("------------------------------------------------------------------------------")
    print("------------------------------------------------------------------------------")

  csv_dic={
      "model_name"  : model_name,
      "tunning_type" : tunning_type,
      "accurecy"    : accuracy, 
      "scaled_accuracy" :scaled_accuracy,
      "feature_selection_method" : feature_selection_method,
      "number_of_features" : number_of_features
  }


  df = pd.DataFrame.from_dict(csv_dic)
  df.to_csv("/content/drive/MyDrive/Iqram Sir/Tinnitus/Features_CSV/"+feat_selection_method+"_"+str(number_of_feat)+"_"+str(model)+".csv",index = False)
  return "Tunning Data Report is Secured for this phase!!!"


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Starting

In [199]:
dataset = pd.read_csv("/content/drive/MyDrive/Iqram Sir/Tinnitus/Tinnitus_106 feature_ALR_EEG_2023.02.08_OnlyGap_No_n0p1p3.csv")
print(dataset.shape)
target = "Class"

#Encoding
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
dataset[target]=encoder.fit_transform(dataset[target])

#dataset = divide_dataset(dataset,0.04611,1)
#print(dataset.shape)

(165, 77)


###Spliting into X and y

In [200]:
X = dataset.iloc[:,3:]
y = dataset[target]
col=list(dataset.iloc[0:0,3:])
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165 entries, 0 to 164
Data columns (total 74 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   n1_gap_peak_ms_20ms           165 non-null    int64  
 1   p2_gap_peak_ms_20ms           165 non-null    int64  
 2   n2_gap_peak_ms_20ms           165 non-null    int64  
 3   n1p2_interval_gap_20ms        165 non-null    int64  
 4   n1n2_interval_gap_20ms        165 non-null    int64  
 5   p2n2_interval_gap_20ms        165 non-null    int64  
 6   n1_gap_peak_amp_20ms          165 non-null    float64
 7   p2_gap_peak_amp_20ms          165 non-null    float64
 8   n2_gap_peak_amp_20ms          165 non-null    float64
 9   n1p2_gap_diff_20ms            165 non-null    float64
 10  p2n2_gap_diff_20ms            165 non-null    float64
 11  n1p2_gap_nogap_diff_20ms      165 non-null    float64
 12  p2n2_gap_nogap_diff_20ms      165 non-null    float64
 13  n1p2_

#Feature Selection

In [138]:
number_of_feat = 30

###ANOVA with f classifciation

In [139]:
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
import pandas as pd



fs = SelectKBest(score_func=f_classif, k=5)
fit = fs.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

featureScores = pd.concat([dfcolumns,dfscores],axis=1)

featureScores.columns = ['Best_columns','Score_ANOVA'] 

lyst = featureScores.nlargest(number_of_feat,'Score_ANOVA')

#lyst.to_csv('Filter_Method_ANOVA_with_f_classif.csv')

list_of_feat = list(lyst["Best_columns"])
selection_method = "ANOVA"

###Embedded Method

In [ ]:
from sklearn.linear_model import LassoCV
reg = LassoCV()
reg.fit(X, y)
print("Best alpha using built-in LassoCV: %f" % reg.alpha_)
print("Best score using built-in LassoCV: %f" %reg.score(X,y))
coef = pd.Series(reg.coef_, index = X.columns)

print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

imp_coef = coef.sort_values()

list_of_feat=[]


for i in range(coef.shape[0]):
  if coef[i]!=0:
    list_of_feat.append(dataset.iloc[:0,i+3].name)
    
df = pd.DataFrame(list_of_feat, columns=['Best_Features'])

#df.to_csv("Embedded_Method.csv")

list_of_feat = list(df["Best_Features"])
if number_of_feat < len(list_of_feat):
  list_of_feat = list_of_feat[:number_of_feat]
selection_method = "Embedded"

Best alpha using built-in LassoCV: 1664496.378245
Best score using built-in LassoCV: 0.000418
Lasso picked 4 variables and eliminated the other 45 variables


###Pearson's with f regression

In [ ]:
from sklearn.datasets import make_regression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
import pandas as pd


fs = SelectKBest(score_func=f_regression, k=5)
fit = fs.fit(X,y)

dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
featureScores = pd.concat([dfcolumns,dfscores],axis=1)

featureScores.columns = ['Best_columns','Score_pearsons'] 


lyst = featureScores.nlargest(number_of_feat,'Score_pearsons')

#lyst.to_csv('Filter_Method_Pearson’s_with_f_regression.csv')

list_of_feat = list(lyst["Best_columns"])
selection_method = "Pearson"

###Sequential Feature Selection

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
sfs = SequentialFeatureSelector(knn, n_features_to_select=number_of_feat)
sfs.fit(X, y)
list_of_feat=[]
list_of_feat=list(sfs.get_feature_names_out(X.columns))

df = pd.DataFrame(list_of_feat, columns=['Best_Features'])

#df.to_csv("Filter_Method_Sequential_feat_Selection_KNN.csv")

list_of_feat = list(df["Best_Features"])
if number_of_feat < len(list_of_feat):
  list_of_feat = list_of_feat[:number_of_feat]

selection_method = "Sequential"

#Feature list

In [140]:
X=dataset[list_of_feat]
y=dataset[target]
print(X.shape)

(165, 17)


In [202]:
list_of_feat2=[   
'p2_gap_peak_amp_50ms',
'p2n2_gap_diff_20ms',
'p2n2_gap_diff_50ms',
'p2n2_interval_gap_50ms',
'n2_gap_peak_amp_20ms',
'gap_auc_n1p2n2_by_total_20ms',
'n1_gap_peak_ms_20ms',
'gap_auc_n1p2n2_by_total_50ms',
'n1p2_gap_no_gap_ratio_50ms',
'gap_auc_p2n2_UP_20ms',
'gap_auc_p2n2_UP_50ms',
'n2_gap_peak_ms_50ms',
'gap_auc_n1p2n2_20ms',
'n1p2_gap_diff_50ms',
'gap_auc_total_abs_20ms',
'n1_gap_peak_ms_50ms',
'n1p2_gap_no_gap_ratio_20ms',
'alpha_peak_auc_50ms',
'gamma_peak_freq_50ms',
'ABR_peak_auc_50ms'
    ]


list_of_feat3=[  
'p2n2_gap_diff_50ms',
'p2_gap_peak_amp_50ms',
'gap_auc_n1p2n2_by_total_20ms',
'gap_auc_n1p2n2_by_total_50ms',
'p2n2_gap_diff_20ms',
'n1_gap_peak_ms_20ms',
'n2_gap_peak_amp_20ms',
'gap_auc_n1p2n2_50ms',
'gap_auc_n1p2_UP_50ms',
'gap_auc_n1p2_LO_50ms',
'gap_auc_n1p2n2_20ms',
'n1_gap_peak_ms_50ms',
'gap_auc_p2n2_UP_50ms',
'n1p2_gap_no_gap_ratio_20ms',
'p2_gap_peak_amp_20ms',
'alpha_peak_auc_50ms',
'gamma_peak_freq_20ms'
    ]

selection_method="list_of_feat3"

number_of_feat = len(list_of_feat3)


X=dataset[list_of_feat3]
y=dataset[target]

In [203]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165 entries, 0 to 164
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   p2n2_gap_diff_50ms            165 non-null    float64
 1   p2_gap_peak_amp_50ms          165 non-null    float64
 2   gap_auc_n1p2n2_by_total_20ms  165 non-null    float64
 3   gap_auc_n1p2n2_by_total_50ms  165 non-null    float64
 4   p2n2_gap_diff_20ms            165 non-null    float64
 5   n1_gap_peak_ms_20ms           165 non-null    int64  
 6   n2_gap_peak_amp_20ms          165 non-null    float64
 7   gap_auc_n1p2n2_50ms           165 non-null    float64
 8   gap_auc_n1p2_UP_50ms          165 non-null    float64
 9   gap_auc_n1p2_LO_50ms          165 non-null    float64
 10  gap_auc_n1p2n2_20ms           165 non-null    float64
 11  n1_gap_peak_ms_50ms           165 non-null    int64  
 12  gap_auc_p2n2_UP_50ms          165 non-null    float64
 13  n1p2_

#Tunning for Scalling and Data Balancing

##Smote : NO
##Scaling : NO

In [204]:
def tune_1():
  X_new = X
  y_new = y
  X_train=X_new.iloc[:133,:]  
  X_test=X_new.iloc[133:,:]
  y_train=y_new.iloc[:133]
  y_test=y_new.iloc[133:]
  
  return X_train,X_test,y_train,y_test
  #return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Smote : NO
##Scaling : All Data (Standard)

In [205]:
def tune_2():
  X_new = X
  y_new = y
  X_train=StandardScaleData(X_new.iloc[:133,:])
  X_test=StandardScaleData(X_new.iloc[133:,:])
  y_train=y_new.iloc[:133]
  y_test=y_new.iloc[133:]
  
  return X_train,X_test,y_train,y_test
  #return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Smote : NO
##Scaling : All Data (MinMax)

In [206]:
def tune_3():
  X_new = X
  y_new = y
  
  X_train=MinMaxScaleData(X_new.iloc[:133,:]) 
  X_test=MinMaxScaleData(X_new.iloc[133:,:])
  y_train=y_new.iloc[:133]
  y_test=y_new.iloc[133:]
  
  return X_train,X_test,y_train,y_test
  #return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Smote : NO
##Scaling : Train (Stadard)

In [207]:
def tune_4():
  X_new = X
  y_new = y
  X_train=X_new.iloc[:133,:]  
  X_test=X_new.iloc[133:,:]
  y_train=y_new.iloc[:133]
  y_test=y_new.iloc[133:]
  
  X_train = StandardScaleData(X_train)
  return X_train, X_test, y_train, y_test

##Smote : NO
##Scaling : Train (MinMax)

In [208]:

def tune_5():
  X_new = X
  y_new = y
  X_train=X_new.iloc[:133,:]  
  X_test=X_new.iloc[133:,:]
  y_train=y_new.iloc[:133]
  y_test=y_new.iloc[133:]
  
  X_train = MinMaxScaleData(X_train)
  return X_train, X_test, y_train, y_test

##Smote : ALL
##Scaling : NO

In [209]:

def tune_6():
  X_new,y_new = Xy_balance(X,y)  
  X_train=X_new.iloc[:133,:]  
  X_test=X_new.iloc[133:,:]
  y_train=y_new.iloc[:133]
  y_test=y_new.iloc[133:]
  
  return X_train,X_test,y_train,y_test
  #return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)


##Smote : Train
##Scaling : NO

In [210]:

def tune_7():
  X_new = X
  y_new = y
  
  X_train=X_new.iloc[:133,:]  
  X_test=X_new.iloc[133:,:]
  y_train=y_new.iloc[:133]
  y_test=y_new.iloc[133:]
  

  X_train,y_train = Xy_balance(X_train,y_train)
  return X_train, X_test, y_train, y_test


##Smote : Train
##Scaling : Train (Standard)

In [211]:
def tune_8():
  X_new = X
  y_new = y
  X_train=X_new.iloc[:133,:]  
  X_test=X_new.iloc[133:,:]
  y_train=y_new.iloc[:133]
  y_test=y_new.iloc[133:]
  
  X_train,y_train = Xy_balance(X_train,y_train)
  X_train = StandardScaleData(X_train)
  return X_train, X_test, y_train, y_test

##Smote : Train
##Scaling : Train (MixMax)

In [212]:
def tune_9():
  X_new = X
  y_new = y
  X_train=X_new.iloc[:133,:]  
  X_test=X_new.iloc[133:,:]
  y_train=y_new.iloc[:133]
  y_test=y_new.iloc[133:]
  
  X_train,y_train = Xy_balance(X_train,y_train)
  X_train = MinMaxScaleData(X_train)
  return X_train, X_test, y_train, y_test

##Smote : All
##Scaling : Train (Standard)

In [213]:
def tune_10():
  X_new,y_new = Xy_balance(X,y)
  X_train=X_new.iloc[:133,:]  
  X_test=X_new.iloc[133:,:]
  y_train=y_new.iloc[:133]
  y_test=y_new.iloc[133:]
  
  X_train = StandardScaleData(X_train)
  return X_train, X_test, y_train, y_test 

##Smote : All
##Scaling : Train (MixMax)

In [214]:
def tune_11():
  X_new,y_new = Xy_balance(X,y)
  X_train=X_new.iloc[:133,:]  
  X_test=X_new.iloc[133:,:]
  y_train=y_new.iloc[:133]
  y_test=y_new.iloc[133:]
  
  X_train = MinMaxScaleData(X_train)
  return X_train, X_test, y_train, y_test 

##Smote : All
##Scaling : All (Standard)

In [215]:
def tune_12():
  X_new,y_new = Xy_balance(X,y)
  X_train=StandardScaleData(X_new.iloc[:133,:])
  X_test=StandardScaleData(X_new.iloc[133:,:])
  y_train=y_new.iloc[:133]
  y_test=y_new.iloc[133:]
  return X_train, X_test, y_train, y_test

##Smote : All
##Scaling : ALL (MinMax)

In [216]:
def tune_13():
  X_new,y_new = Xy_balance(X,y)
  X_train=MinMaxScaleData(X_new.iloc[:133,:])
  X_test=MinMaxScaleData(X_new.iloc[133:,:])
  y_train=y_new.iloc[:133]
  y_test=y_new.iloc[133:]
  return X_train, X_test, y_train, y_test

#  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Tunn Dictionary

In [217]:
tune_dic={
  "Smote : NO    ; Scaling : NO"                  : tune_1(),
  "Smote : NO    ; Scaling : All Data(Standard)"  : tune_2(),
  "Smote : NO    ; Scaling : All Data(MinMax)"    : tune_3(),
  #"Smote : NO    ; Scaling : Train(Standard)"     : tune_4(),
  #"Smote : NO    ; Scaling : Train(MinMax)"       : tune_5(),
  "Smote : ALL   ; Scaling : NO"                  : tune_6(),
  "Smote : Train ; Scaling : NO"                  : tune_7(),
  #"Smote : Train ; Scaling : Train(Standard)"     : tune_8(),
  #"Smote : Train ; Scaling : Train(MinMax)"       : tune_9(),
  #"Smote : All   ; Scaling : Train(Standard)"     : tune_10(),
  #"Smote : All   ; Scaling : Train(MinMax)"       : tune_11(),
  "Smote : All   ; Scaling : All(Standard)"       : tune_12(),
  "Smote : All   ; Scaling : All(MinMax)"        : tune_13()
}



---



# **Training**

##ADABOOST

In [218]:
%%time
from sklearn.ensemble import AdaBoostClassifier
ada_defult = AdaBoostClassifier(random_state=0)
tune_report_csv(ada_defult,tune_dic,selection_method,number_of_feat)


Smote : NO    ; Scaling : NO
Accurecy:  0.59
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.47
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.62
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.62
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.53
------------------------------------------------------------------------------
----------------------------------

'Tunning Data Report is Secured for this phase!!!'



---



---



##Graddient Boosting

In [219]:
%%time
from sklearn.ensemble import GradientBoostingClassifier
gradBoost_default = GradientBoostingClassifier(random_state=0)

tune_report_csv(gradBoost_default,tune_dic,selection_method,number_of_feat)

Smote : NO    ; Scaling : NO
Accurecy:  0.59
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.5
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.59
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.64
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.59
------------------------------------------------------------------------------
-----------------------------------

'Tunning Data Report is Secured for this phase!!!'

###Histogram-Based Gradient Boosting

In [220]:
%%time
from sklearn.ensemble import HistGradientBoostingClassifier

hisgradBoost_default = HistGradientBoostingClassifier(random_state=0)

tune_report_csv(hisgradBoost_default,tune_dic,selection_method,number_of_feat)

Smote : NO    ; Scaling : NO
Accurecy:  0.66
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.53
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.56
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.69
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.66
------------------------------------------------------------------------------
----------------------------------

'Tunning Data Report is Secured for this phase!!!'



---



---



##Random Forest

In [221]:
%%time
RanFor={}

from sklearn.ensemble import RandomForestClassifier
rf_default = RandomForestClassifier(random_state=0,n_jobs=-1)

tune_report_csv(rf_default,tune_dic,selection_method,number_of_feat)

Smote : NO    ; Scaling : NO
Accurecy:  0.62
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.59
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.72
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.67
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.62
------------------------------------------------------------------------------
----------------------------------

'Tunning Data Report is Secured for this phase!!!'



---



---



##XGB

In [222]:
%%time
import xgboost as xgb
xgb_deafult = xgb.XGBClassifier(random_state=0,n_jobs=-1)

tune_report_csv(xgb_deafult,tune_dic,selection_method,number_of_feat)

Smote : NO    ; Scaling : NO
Accurecy:  0.56
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.47
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.56
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.62
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.56
------------------------------------------------------------------------------
----------------------------------

'Tunning Data Report is Secured for this phase!!!'

##KNN

In [223]:
%%time
from sklearn.neighbors import KNeighborsClassifier
knn_default = KNeighborsClassifier(n_jobs = -1)

tune_report_csv(knn_default,tune_dic,selection_method,number_of_feat)

Smote : NO    ; Scaling : NO
Accurecy:  0.47
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.66
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.59
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.51
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.44
------------------------------------------------------------------------------
----------------------------------

'Tunning Data Report is Secured for this phase!!!'

##NB

In [224]:
%%time
from sklearn.naive_bayes import GaussianNB

nb_deafult = GaussianNB()

tune_report_csv(nb_deafult,tune_dic,selection_method,number_of_feat)

Smote : NO    ; Scaling : NO
Accurecy:  0.72
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.72
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.56
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.69
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.72
------------------------------------------------------------------------------
----------------------------------

'Tunning Data Report is Secured for this phase!!!'

##Tree algorithms

###DecisionTreeRegressor

In [225]:
%%time
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()
tune_report_csv(dtc,tune_dic,selection_method,number_of_feat)

Smote : NO    ; Scaling : NO
Accurecy:  0.56
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.53
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.41
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.67
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.47
------------------------------------------------------------------------------
----------------------------------

'Tunning Data Report is Secured for this phase!!!'

In [226]:
!pip install catboost

from catboost import CatBoostClassifier
#cat = CatBoostClassifier(task_type="GPU")
cat=CatBoostClassifier()

tune_report_csv(cat,tune_dic,selection_method,number_of_feat)

Streaming output truncated to the last 5000 lines.
24:	learn: 0.6416396	total: 46.1ms	remaining: 1.8s
25:	learn: 0.6398046	total: 47.7ms	remaining: 1.79s
26:	learn: 0.6385225	total: 49.3ms	remaining: 1.78s
27:	learn: 0.6366770	total: 51ms	remaining: 1.77s
28:	learn: 0.6345691	total: 52.6ms	remaining: 1.76s
29:	learn: 0.6329616	total: 54.3ms	remaining: 1.75s
30:	learn: 0.6314452	total: 55.9ms	remaining: 1.75s
31:	learn: 0.6295537	total: 57.6ms	remaining: 1.74s
32:	learn: 0.6278329	total: 59.2ms	remaining: 1.73s
33:	learn: 0.6254811	total: 60.8ms	remaining: 1.73s
34:	learn: 0.6239025	total: 62.5ms	remaining: 1.72s
35:	learn: 0.6223540	total: 64.1ms	remaining: 1.72s
36:	learn: 0.6204807	total: 65.7ms	remaining: 1.71s
37:	learn: 0.6187626	total: 67.3ms	remaining: 1.7s
38:	learn: 0.6171348	total: 68.9ms	remaining: 1.7s
39:	learn: 0.6152093	total: 70.6ms	remaining: 1.69s
40:	learn: 0.6142551	total: 72.2ms	remaining: 1.69s
41:	learn: 0.6126343	total: 73.9ms	remaining: 1.68s
42:	learn: 0.61098

'Tunning Data Report is Secured for this phase!!!'

#Result Analysis

In [227]:
import pandas as pd
import os
import glob


csv_files = glob.glob(os.path.join("/content/drive/MyDrive/Iqram Sir/Tinnitus/Features_CSV", "*.csv"))

final_dataset = pd.DataFrame()

for f in csv_files:
            df_temp = pd.read_csv(f)
            final_dataset = final_dataset.append(df_temp, ignore_index=True)


##Sorting according to accuracy

In [228]:
final_dataset = final_dataset.sort_values("accurecy",ascending=False,kind="mergesort")
final_dataset = final_dataset.reset_index(drop = True)
final_dataset.head(10)

,model_name,tunning_type,accurecy,scaled_accuracy,feature_selection_method,number_of_features
0,KNeighborsClassifier(n_jobs=-1),Smote : All ; Scaling : All(Standard),0.794872,0.79,list_of_feat2,20
1,KNeighborsClassifier(n_jobs=-1),Smote : All ; Scaling : All(MinMax),0.769231,0.77,list_of_feat2,20
2,GaussianNB(),Smote : All ; Scaling : All(Standard),0.743590,0.74,list_of_feat2,20
3,AdaBoostClassifier(random_state=0),Smote : Train ; Scaling : NO,0.718750,0.72,list_of_feat2,20
4,GaussianNB(),Smote : NO ; Scaling : NO,0.718750,0.72,list_of_feat2,20
5,GaussianNB(),Smote : Train ; Scaling : NO,0.718750,0.72,list_of_feat2,20
6,"RandomForestClassifier(n_jobs=-1, random_state=0)",Smote : NO ; Scaling : All Data(MinMax),0.718750,0.72,list_of_feat3,17
7,GaussianNB(),Smote : NO ; Scaling : NO,0.718750,0.72,list_of_feat3,17
8,GaussianNB(),Smote : NO ; Scaling : All Data(Standard),0.718750,0.72,list_of_feat3,17
9,GaussianNB(),Smote : Train ; Scaling : NO,0.718750,0.72,list_of_feat3,17


##Saving

In [137]:
final_dataset.to_csv("/content/drive/MyDrive/Iqram Sir/Tinnitus/FSR.csv",index=False)

In [229]:
final_dataset.to_csv("FSR_list_2_3.csv",index=False)

In [99]:
final_dataset.head(10)

,model_name,tunning_type,accurecy,scaled_accuracy,feature_selection_method,number_of_features
0,<catboost.core.CatBoostClassifier object at 0x...,Smote : ALL ; Scaling : NO,0.828571,0.83,given_list_2,20
1,HistGradientBoostingClassifier(random_state=0),Smote : ALL ; Scaling : NO,0.800000,0.80,given_list_2,20
2,HistGradientBoostingClassifier(random_state=0),Smote : All ; Scaling : All(Standard),0.800000,0.80,given_list_2,20
3,"RandomForestClassifier(n_jobs=-1, random_state=0)",Smote : ALL ; Scaling : NO,0.800000,0.80,given_list_2,20
4,"RandomForestClassifier(n_jobs=-1, random_state=0)",Smote : NO ; Scaling : All Data(MinMax),0.787879,0.79,given_list_2,20
5,DecisionTreeClassifier(),Smote : NO ; Scaling : All Data(Standard),0.787879,0.79,given_list_2,20
6,AdaBoostClassifier(random_state=0),Smote : All ; Scaling : All(MinMax),0.771429,0.77,given_list_2,20
7,XGBClassifier(n_jobs=-1),Smote : All ; Scaling : All(MinMax),0.771429,0.77,given_list_2,20
8,KNeighborsClassifier(n_jobs=-1),Smote : All ; Scaling : All(MinMax),0.771429,0.77,given_list_2,20
9,<catboost.core.CatBoostClassifier object at 0x...,Smote : All ; Scaling : All(Standard),0.771429,0.77,given_list_2,20
